In [41]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.model_selection import KFold
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# # List files in your Google Drive
# !ls /content/drive/MyDrive/UCSD_Courses/Data

In [2]:
# Define the path
save_path = '/content/drive/MyDrive/UCSD_Courses/Behavior_Finance/Hedge_fund_project_predicted_value/'

In [3]:
!pip install torch --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 MB 6.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0
    Uninstalling torch-2.3.0:
      Successfully uninstalled torch-2.3.0


In [4]:
# df = pd.read_feather("/content/drive/MyDrive/UCSD_Courses/Data/df_daily_technical_indicators.feather")
df = pd.read_feather("df_daily_technical_indicators.feather")

In [5]:
def deciles(inser):
    outser = pd.qcut(inser, q=10, labels=range(10))
    return outser

In [7]:
# generate a label if stock return > median. label = 1; else label = 0
# when calculating median, eliminate meanless data points from padding value
# median_return = df[df['Future 30 days return'] != 0].groupby('date')['Future 30 days return'].median()
# df['median_return'] = df.index.get_level_values('date').map(median_return)
# df['label'] = (df['Future 30 days return'] > df['median_return']).astype(int)
df['label'] = df.groupby('date')['Future 30 days return'].apply(deciles).reset_index(level=0, drop=True)

In [8]:
df

Bollinger Band Percent        rsi  MACD_12_26_9  \
PERMNO date                                                          
10001  1986-02-26                0.895285  69.377403      0.009111   
       1986-02-27                0.851887  69.377403      0.009368   
       1986-02-28                0.979186  73.918841      0.010152   
       1986-03-03                0.590435  56.023616      0.009341   
       1986-03-04                0.573235  56.023616      0.008440   
...                                   ...        ...           ...   
93436  2023-11-09                0.492374  46.000779     -0.036636   
       2023-11-10                0.346916  38.538966     -0.039103   
       2023-11-13                0.438164  42.411998     -0.036350   
       2023-11-14                0.631862  49.111857     -0.029819   
       2023-11-15                0.998316  57.217313     -0.019441   

                   MACDh_12_26_9  MACDs_12_26_9  Momentum       OBV  \
PERMNO date                                                           
10001  1986-02-26       0.000926       0.008185  0.030612 -0.017736   
       1986-02-27       0.000946       0.008421  0.030612 -0.025592   
       1986-02-28       0.001450       0.008701  0.030303  0.768732   
       1986-03-03       0.000372       0.008968 -0.009901 -0.380366   
       1986-03-04      -0.000423       0.008863  0.000000  0.016461   
...                          ...            ...       ...       ...   
93436  2023-11-09       0.008824      -0.045460  0.047293  0.002503   
       2023-11-10       0.007186      -0.046290 -0.030205 -0.048671   
       2023-11-13       0.007146      -0.043496  0.010498  0.014638   
       2023-11-14       0.009532      -0.039351  0.087238  0.009032   
       2023-11-15       0.014112      -0.033553  0.145248 -0.014639   

                         ATR  Future 30 days return Size  lag vwretx  \
PERMNO date                                                            
10001  1986-02-26  16.428309               0.000000    2   -0.001791   
       1986-02-27  15.141755               0.000000    2    0.001307   
       1986-02-28  25.018615              -0.009804    3    0.010952   
       1986-03-03  -3.246377               0.010000    2    0.001845   
       1986-03-04  -4.666667               0.010000    1   -0.003870   
...                      ...                    ...  ...         ...   
93436  2023-11-09   0.405085               0.145829   10   -0.000455   
       2023-11-10  -0.002893               0.202686   10   -0.008586   
       2023-11-13  -0.036597               0.195481   10    0.013823   
       2023-11-14   0.342494               0.168656   10   -0.000386   
       2023-11-15   0.749947               0.066425   10    0.022639   

                   lag ewretx  lag sprtrn  median_return label  
PERMNO date                                                     
10001  1986-02-26    0.000788   -0.002452       0.042313     3  
       1986-02-27    0.003222    0.001117       0.046225     3  
       1986-02-28    0.007195    0.012185       0.035714     3  
       1986-03-03    0.005390    0.000661       0.037037     4  
       1986-03-04    0.000720   -0.006610       0.037736     4  
...                       ...         ...            ...   ...  
93436  2023-11-09   -0.006451    0.001005       0.139057     5  
       2023-11-10   -0.014880   -0.008084       0.158942     6  
       2023-11-13    0.004011    0.015616       0.155249     6  
       2023-11-14    0.000257   -0.000836       0.155214     5  
       2023-11-15    0.030881    0.019075       0.106725     3  

[32477610 rows x 15 columns]

In [9]:
# Convert DataFrame to numeric, forcing non-numeric values to NaNs
df = df.apply(pd.to_numeric, errors='coerce')
df = df.replace([np.inf, -np.inf], np.nan).fillna(0)

In [10]:
# Calculate the cumulative data the stock data got!
df['data_cumcount'] = df.groupby('PERMNO').cumcount()

In [11]:
# define the look_back_period as the time series length the model trained on
look_back_period = 100
begin_date = '2015-01-01'
end_date = '2016-12-31'
train_test_split_date = '2016-01-01'
date = df.index.get_level_values('date')
df_temp = df[(date>begin_date)&(date<=end_date)].copy()
del date
# We only want data point with enough lag information
df_temp = df_temp[df_temp['data_cumcount'] > look_back_period]
# eliminate meanless data points from padding value
df_temp = df_temp[df_temp['Future 30 days return'] != 0]
df_temp = df_temp.reset_index()

# Generate Pair_PERMNO_date for future indexing
df_temp['Pair_PERMNO_date'] = list(zip(df_temp['PERMNO'], df_temp['date']))
# take the last day in the month as sample
df_temp['date'] = pd.to_datetime(df_temp['date']).dt.to_period('M').astype(str)
df_temp = df_temp.groupby(['PERMNO','date']).tail(1)

# reset index to make it more convenient for future use
df_temp = df_temp.reset_index()

In [12]:
df_temp

,index,PERMNO,date,Bollinger Band Percent,rsi,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Momentum,OBV,ATR,Future 30 days return,Size,lag vwretx,lag ewretx,lag sprtrn,median_return,label,data_cumcount,Pair_PERMNO_date
0,17,10001,2015-01,0.144353,29.700467,-0.023386,-0.002639,-0.020747,-0.071038,-0.414548,-11.875583,-0.034314,3,-0.010010,-0.001501,-0.013388,0.017212,2,5367,"(10001, 2015-01-28)"
1,27,10001,2015-02,0.750474,50.379245,-0.005768,0.007277,-0.013045,0.068783,0.036489,6.318367,-0.011881,2,-0.001683,-0.000319,-0.001476,0.009928,3,5377,"(10001, 2015-02-27)"
2,46,10001,2015-03,0.807974,55.237872,-0.001508,0.003042,-0.004550,0.015228,0.214805,11.005684,0.001000,2,0.011441,0.006734,0.012237,-0.010810,5,5396,"(10001, 2015-03-31)"
3,67,10001,2015-04,0.640162,51.876286,0.002436,0.001217,0.001219,0.004016,-0.084449,8.591645,-0.011000,1,-0.004006,-0.005035,-0.003740,0.007519,3,5417,"(10001, 2015-04-30)"
4,87,10001,2015-05,0.542303,52.150620,0.002739,-0.000648,0.003387,0.001984,0.119506,-0.671347,-0.009901,1,-0.001340,-0.001135,-0.001267,-0.007921,4,5437,"(10001, 2015-05-29)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52650,1050263,93436,2016-08,-0.206755,31.984428,-0.009318,-0.007537,-0.001781,-0.063251,-0.053368,-1.166506,-0.046513,10,-0.001643,-0.000446,-0.001954,-0.011100,2,883,"(93436, 2016-08-31)"
52651,1050284,93436,2016-09,0.413642,39.700502,-0.012351,0.004837,-0.017188,0.023718,0.013172,-0.058491,-0.076482,10,-0.009267,-0.008515,-0.009321,0.014236,1,904,"(93436, 2016-09-30)"
52652,1050305,93436,2016-10,0.397155,46.215030,-0.005828,0.002717,-0.008545,-0.007642,-0.055305,0.163188,-0.037706,10,-0.002635,-0.003908,-0.003108,0.135324,1,925,"(93436, 2016-10-31)"
52653,1050326,93436,2016-11,0.525233,46.580455,-0.008084,0.006674,-0.014758,0.022768,0.020850,0.361069,0.211109,10,0.000770,-0.001947,0.001335,0.025851,9,946,"(93436, 2016-11-30)"


In [16]:
df_train = df_temp[df_temp['date']<train_test_split_date]
df_test = df_temp[df_temp['date']>=train_test_split_date]

In [17]:
label_name = 'label'
X_train = df_train.drop(label_name, axis=1)
X_test = df_test.drop(label_name, axis=1)

y_train = df_train[label_name]
y_test = df_test[label_name]

In [18]:
import torch.nn.functional as F
# F.one_hot(torch.tensor(0), num_classes=10)

In [32]:
df.index.get_loc(df_temp['Pair_PERMNO_date'][0])

5367

In [36]:
F.one_hot(torch.tensor(df.iloc[5367]['label'].tolist(), dtype=torch.int64), num_classes=10)

tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [37]:
class TimeSeriesDataset(Dataset):
    def __init__(self, whole_data, X, y=None, augment=True):
        self.whole_data = whole_data
        self.X = X #torch.tensor(X, dtype=torch.float32)
        self.y = y if y is not None else None #torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):


        if self.y is not None:
          x = []
          y = []

          permo_date_locator = df.index.get_loc(df_temp['Pair_PERMNO_date'][idx])
          #get x given index
          x_temp  = df.iloc[permo_date_locator-look_back_period+1:permo_date_locator+1].drop(['Future 30 days return', 'label', 'data_cumcount'], axis = 1).to_numpy()
          x.append(x_temp)
          #get y given index
          y_temp = F.one_hot(torch.tensor(df.iloc[permo_date_locator]['label'], dtype=torch.int64), num_classes=10)
          y.append(y_temp)

          x = torch.tensor(x_temp, dtype=torch.float32)
          y = torch.tensor(y_temp, dtype=torch.float32)
          return x, y

        else:
          x = []
          i = idx
          permo_date_locator = df.index.get_loc(df_temp['Pair_PERMNO_date'][idx])
          #get x given index
          x_temp  = df.iloc[permo_date_locator-look_back_period+1:permo_date_locator+1].drop(['Future 30 days return', 'label', 'data_cumcount'], axis = 1).to_numpy()
          x.append(x_temp)

          x = torch.tensor(x_temp, dtype=torch.float32)

          return x

        '''
    def get_time_series_sample(self, x):
        return x
        '''

In [20]:
df

Bollinger Band Percent        rsi  MACD_12_26_9  \
PERMNO date                                                          
10001  1986-02-26                0.895285  69.377403      0.009111   
       1986-02-27                0.851887  69.377403      0.009368   
       1986-02-28                0.979186  73.918841      0.010152   
       1986-03-03                0.590435  56.023616      0.009341   
       1986-03-04                0.573235  56.023616      0.008440   
...                                   ...        ...           ...   
93436  2023-11-09                0.492374  46.000779     -0.036636   
       2023-11-10                0.346916  38.538966     -0.039103   
       2023-11-13                0.438164  42.411998     -0.036350   
       2023-11-14                0.631862  49.111857     -0.029819   
       2023-11-15                0.998316  57.217313     -0.019441   

                   MACDh_12_26_9  MACDs_12_26_9  Momentum       OBV  \
PERMNO date                                                           
10001  1986-02-26       0.000926       0.008185  0.030612 -0.017736   
       1986-02-27       0.000946       0.008421  0.030612 -0.025592   
       1986-02-28       0.001450       0.008701  0.030303  0.768732   
       1986-03-03       0.000372       0.008968 -0.009901 -0.380366   
       1986-03-04      -0.000423       0.008863  0.000000  0.016461   
...                          ...            ...       ...       ...   
93436  2023-11-09       0.008824      -0.045460  0.047293  0.002503   
       2023-11-10       0.007186      -0.046290 -0.030205 -0.048671   
       2023-11-13       0.007146      -0.043496  0.010498  0.014638   
       2023-11-14       0.009532      -0.039351  0.087238  0.009032   
       2023-11-15       0.014112      -0.033553  0.145248 -0.014639   

                         ATR  Future 30 days return  Size  lag vwretx  \
PERMNO date                                                             
10001  1986-02-26  16.428309               0.000000     2   -0.001791   
       1986-02-27  15.141755               0.000000     2    0.001307   
       1986-02-28  25.018615              -0.009804     3    0.010952   
       1986-03-03  -3.246377               0.010000     2    0.001845   
       1986-03-04  -4.666667               0.010000     1   -0.003870   
...                      ...                    ...   ...         ...   
93436  2023-11-09   0.405085               0.145829    10   -0.000455   
       2023-11-10  -0.002893               0.202686    10   -0.008586   
       2023-11-13  -0.036597               0.195481    10    0.013823   
       2023-11-14   0.342494               0.168656    10   -0.000386   
       2023-11-15   0.749947               0.066425    10    0.022639   

                   lag ewretx  lag sprtrn  median_return  label  data_cumcount  
PERMNO date                                                                     
10001  1986-02-26    0.000788   -0.002452       0.042313      3              0  
       1986-02-27    0.003222    0.001117       0.046225      3              1  
       1986-02-28    0.007195    0.012185       0.035714      3              2  
       1986-03-03    0.005390    0.000661       0.037037      4              3  
       1986-03-04    0.000720   -0.006610       0.037736      4              4  
...                       ...         ...            ...    ...            ...  
93436  2023-11-09   -0.006451    0.001005       0.139057      5           2693  
       2023-11-10   -0.014880   -0.008084       0.158942      6           2694  
       2023-11-13    0.004011    0.015616       0.155249      6           2695  
       2023-11-14    0.000257   -0.000836       0.155214      5           2696  
       2023-11-15    0.030881    0.019075       0.106725      3           2697  

[32477610 rows x 16 columns]

In [38]:
# Create dataset
dataset = TimeSeriesDataset(df, X_train, y_train)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
len(next(iter(dataloader)))

2

# Building my NN Model

In [22]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
seed = 42
device

device(type='mps')

In [23]:
# Define a simpler Model
class SimpleConvEncoder(nn.Module):
    def __init__(self, T, C):
        super(SimpleConvEncoder, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=C, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change shape to [B, C, T]
        x = F.relu(self.conv1(x))
        x = self.dropout(x)
        x = F.relu(self.conv2(x))
        x = self.dropout(x)
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        return x

# Define a simple Classification Head
class SimpleClassificationHead(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(SimpleClassificationHead, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 4)
        self.fc3 = nn.Linear(hidden_dim // 4, 10)  # Output 1 node for binary classification
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Combine Encoder and Classification Head into one model
class SimpleTimeSeriesModel(nn.Module):
    def __init__(self, T, C, hidden_dim):
        super(SimpleTimeSeriesModel, self).__init__()
        self.encoder = SimpleConvEncoder(T, C)
        self.classification_head = SimpleClassificationHead(T * 256, hidden_dim)  # Update input dim based on the flattened size
        self.loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, x, ybatch=None):
        x = self.encoder(x)  # x shape will be [B, T * 256] after flattening
        logits = self.classification_head(x).squeeze(-1)  # logits shape will be [B]
        loss = None
        if ybatch is not None:
            loss = self.loss_fn(logits, ybatch.float())

        return logits, loss


In [24]:
# Early stopping mechanism
class EarlyStopping_Train:
    def __init__(self, patience=10, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.train_loss_min = np.Inf
        self.delta = delta

    def __call__(self, train_loss, model):
        score = -train_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(train_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'Training loss did not improve for {self.counter} epochs')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(train_loss, model)
            self.counter = 0

    def save_checkpoint(self, train_loss, model):
        if self.verbose:
            print(f'Training loss decreased ({self.train_loss_min:.6f} --> {train_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), 'checkpoint.pth')
        self.train_loss_min = train_loss

In [39]:
label_name = 'label'
X = df_train.drop(label_name, axis=1)
X_test = df_test.drop(label_name, axis=1)

y = df_train[label_name]
y_test = df_test[label_name]


# Parameters
T = look_back_period
C = df_temp.shape[1]-8
n_embd = T*C
max_seq_length = n_embd
n_layer = 2 # 6
n_head = 4 # 4
n_hidden = 1000 # 1000


batch_size = 128


epochs = 2000#1000
patience = 10
seed = 42
learning_rate = 0.001#
clip_value = 5.0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

# Train the model on the entire training data
train_dataset = TimeSeriesDataset(df, X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

model = SimpleTimeSeriesModel(T, C, n_hidden ).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

epochs = 1000  # Change this value as needed
early_stopping = EarlyStopping_Train(patience=patience, verbose=True)


In [50]:

for epoch in range(epochs):
    model.train()
    running_train_loss = 0.0
    for batch_idx, (xbatch, ybatch) in enumerate(train_loader):
        xbatch, ybatch = xbatch.to(device), ybatch.to(device)
        optimizer.zero_grad()
        out, loss = model(xbatch, ybatch)
        # print(out)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_value)
        optimizer.step()
        running_train_loss += loss.item()
        # break

    avg_train_loss = running_train_loss / len(train_loader)
    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {avg_train_loss:.4f}')
    # break
    # Early stopping
    early_stopping(avg_train_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

tensor([[-1.7503, -2.0977, -2.5557,  ..., -2.1471, -2.1478, -2.6472],
        [-2.4844, -2.3567, -2.2039,  ..., -1.7311, -2.0843, -1.9502],
        [-1.8576, -2.2922, -2.1078,  ..., -2.1968, -1.8404, -1.9151],
        ...,
        [-2.7343, -2.6347, -2.6284,  ..., -2.7131, -2.2426, -2.6448],
        [-1.6914, -2.4425, -2.3417,  ..., -2.1362, -3.0606, -2.4508],
        [-1.8981, -2.6332, -2.6645,  ..., -2.6176, -2.7258, -2.5079]],
       device='mps:0', grad_fn=<SqueezeBackward1>)
Epoch [1/1000], Train Loss: 0.0015


In [60]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, ConfusionMatrixDisplay


# Load the last checkpoint with the best model
# model.load_state_dict(torch.load('checkpoint.pth'))
# Predict on the test data
test_dataset = TimeSeriesDataset(df, X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Evaluate the model
model.eval()
all_preds = []
all_labels = []
all_probs = []
with torch.no_grad():
    for xbatch, ybatch in test_loader:
        xbatch, ybatch = xbatch.to(device), ybatch.to(device)
        out, loss = model(xbatch, ybatch)
        probs = torch.sigmoid(out).cpu().numpy()
        all_probs.extend(probs)
        all_preds.extend(torch.max(out, 1)[1].cpu().numpy())
        all_labels.extend(torch.max(ybatch, 1)[1].cpu().numpy())
        # break


In [61]:
all_probs = np.array(all_probs)
all_preds = np.array(all_preds).astype(int)
all_labels = np.array(all_labels).astype(int)
# all_preds_binary = (all_preds > 0.5).astype(int)

In [63]:
# all_preds

array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])

In [49]:
# all_preds_binary[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [66]:
cm = confusion_matrix(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')
hit_rate = recall_score(all_labels, all_preds, average='weighted')
ppv = precision_score(all_labels, all_preds, average='weighted')

print("Confusion Matrix:")
print(cm)
print(f"F1 Score: {f1:.4f}")
print(f"Hit Rate (Recall): {hit_rate:.4f}")
print(f"Positive Predictive Value (Precision): {ppv:.4f}")

Confusion Matrix:
[[ 0  0  0  0  0  0  6  0  0  0]
 [ 0  0  0  0  0  0 14  0  0  0]
 [ 0  0  0  0  0  0 20  0  0  0]
 [ 0  0  0  0  0  0 13  0  0  0]
 [ 0  0  0  0  0  0 21  0  0  0]
 [ 0  0  0  0  0  0 17  0  0  0]
 [ 0  0  0  0  0  0 14  0  0  0]
 [ 0  0  0  0  0  0  9  0  0  0]
 [ 0  0  0  0  0  0 11  0  0  0]
 [ 0  0  0  0  0  0  3  0  0  0]]
F1 Score: 0.0216
Hit Rate (Recall): 0.1094
Positive Predictive Value (Precision): 0.0120


In [ ]:
# Saving the data
#X_test['p(0)'] = all_probs[:, 0]
X_test['p(1)'] = all_probs
X_test.reset_index()[['PERMNO','date','label','p(1)']].to_feather(save_path+end_date+'.feather')